## Particle Swarm Optmization - The Cornfield Vector

### Modelando a particula:

In [ ]:
import numpy as np

class Particle:
  def __init__(self, dimensions, bounds, id):
    '''
    Construtor
    '''
    #Cada particula tem o seu "melhor" pessoal(individual)
    self.id = id

    self.dimensions = dimensions
    self.bounds = bounds # limites da area 
    self.position = [] # posicao da particula
    self.dist = np.inf # distancia que esta do alvo
    self.velocity = [] # vetor velocidade
    self.pbest_pos = [] # melhor posicao da particula
    self.pbest_dist = np.inf # melhor distancia que a particula conseguiu chegar no obj

    # Definir valores aleatorios de velocidade e posicao.
    for i in range(self.dimensions):
      self.position.append(np.random.uniform(
        self.bounds[i][0], self.bounds[i][1]))
      self.velocity.append(np.random.uniform(
        self.bounds[i][0], self.bounds[i][1]))

  @staticmethod
  def _euclidean_distance(p1_pos, p2_pos):
    '''
    Calcula a distância euclidiana.
    '''
    pp1_pos = np.array(p1_pos)
    pp2_pos = np.array(p2_pos)
    distance = np.sqrt(sum((pp1_pos - pp2_pos)**2))

    return distance

  def evaluate(self, target_pos):
    '''
    Função que avalia e compara a proximidade da partículas em relação ao objetivo. So valia o pbest
    '''
    #Calculara o pbest
    self.dist = Particle._euclidean_distance(self.position, target_pos)
    #Checa se o valor do pbest eh menor que a distancia self.dist, se for sera o novo pbest
    if self.dist < self.pbest_dist:
      self.pbest_dist = self.dist
      self.pbest_pos = self.position

  def update_velocity(self, gbest_pos):
    '''
    Atualiza a velocidade com base no pbest e no gbest.
    '''
    for i in range(0, self.dimensions):
      # pesos de forma aleatoria
      r1 = np.random.uniform(0, 1)
      r2 = np.random.uniform(0, 1)

      vel_cognitive = r1 * (self.pbest_pos[i] - self.position[i]) 
      vel_social = r2 * (gbest_pos[i] - self.position[i])

      self.velocity[i] = (self.velocity[i] + vel_cognitive + vel_social) / 3

  def update_position(self):
    '''
    Atualiza a posição de cada uma das partículas.
    '''
    for i in range(0, self.dimensions):
      self.position[i] = self.position[i] + self.velocity[i]
      # checa se a particula saiu do bound
      if self.position[i] < self.bounds[i][0]:
        self.position[i] = self.bounds[i][0]

      if self.position[i] > self.bounds[i][1]:
        self.position[i] = self.bounds[i][1]


### Código para plotagem do PSO:

In [ ]:
from matplotlib import pyplot as plt
from PIL import Image
import glob
import os
import shutil

class PlotUtils:

  directory = "pso_plots"
  filename = 'pso.gif'

  @staticmethod
  def start_plot():
    if os.path.exists(PlotUtils.directory):
      shutil.rmtree(PlotUtils.directory)
    if not os.path.exists(PlotUtils.directory):
      os.makedirs(PlotUtils.directory)

  @staticmethod
  def plot_particle(particle):
    plt.scatter(particle.position[0], particle.position[1])

  @staticmethod
  def plot_iteration(i, bounds):
    plt.title(f"PSO {i}")
    plt.xlim(bounds[0][0], bounds[0][1])
    plt.ylim(bounds[1][0], bounds[1][1])
    plt.xlabel('x[0]')
    plt.ylabel('x[1]')
    iteration = str(i).zfill(5)
    plt.savefig(
      f"{PlotUtils.directory}/iteration_{iteration}.png", facecolor="white", dpi=75)
    plt.close()

  @staticmethod
  def save():
    images = [Image.open(f) for f in sorted(
      glob.glob(PlotUtils.directory+"/*"))]
    img = images[0]
    img.save(fp=PlotUtils.filename, format='GIF',
              append_images=images, save_all=True, duration=200, loop=0)
    if os.path.exists(PlotUtils.directory):
      shutil.rmtree(PlotUtils.directory)

### Modelagem do enxame:

In [ ]:
class Swarm:

  def __init__(self, particles, target_pos):
    '''
    Construtor
    '''
    self.particles = particles
    self.target_pos = target_pos #posicao do alvo
    self.gbest_pos = [] #melhor posicao global
    self.gbest_dist = np.inf #melhor dist global

  def swarm_evaluate(self):
    '''
    Percorre todas as partículas e avalia/atualiza seu melhor pessoal e o melhor global do enxame.
    '''
    #para cada particula do enxame checara o pbest de cada um
    for p in self.particles:
      p.evaluate(self.target_pos)

      if p.dist < self.gbest_dist:
        self.gbest_pos = p.position
        self.gbest_dist = p.dist

  def swarm_update_velocities(self):
    '''
    Percorre todas as partículas e chama a função para atualizar a velocidade.
    '''
    for p in self.particles:
      p.update_velocity(self.gbest_pos)

  def swarm_update_positions(self):
    '''
    Percorre todas as partículas e chama a função para atualizar a posição.
    '''
    for p in self.particles:
      p.update_position()

### Execução:

In [ ]:
def executar_pso(num_particulas, num_iteracoes,
                  posicao_alvo, num_dimencoes, limites): #num de iteracoes eh o criterio de parada
  PlotUtils.start_plot()
  print("inicialização")

  particles = []
  for i in range(num_particulas):
    particles.append(Particle(num_dimencoes, limites, id=i))

  swarm = Swarm(particles, posicao_alvo)

  print("começando as iterações")
  i = 0
  while i < num_iteracoes:
    
    print(f"iteração {i}")
    '''
    for p in swarm.particles:
      print(f'id{p.id} pbest: {p.pbest_dist}')
      print(f'gbest: {swarm.gbest_dist}')
    '''
    swarm.swarm_evaluate() #valia o pbest e o gbest, respectivamente
    swarm.swarm_update_velocities()
    swarm.swarm_update_positions()

    for p in swarm.particles:
      PlotUtils.plot_particle(p)
    PlotUtils.plot_iteration(i, limites)

    i += 1

  PlotUtils.save()

executar_pso(
  num_particulas = 10,
  num_iteracoes = 20,
  posicao_alvo=[20, -20],
  num_dimencoes = 2,
  limites = [(-50, 50), (-50, 50)])

inicialização
começando as iterações
iteração 0
iteração 1
iteração 2
iteração 3
iteração 4
iteração 5
iteração 6
iteração 7
iteração 8
iteração 9
iteração 10
iteração 11
iteração 12
iteração 13
iteração 14
iteração 15
iteração 16
iteração 17
iteração 18
iteração 19
